In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qureai-headct/prediction_probabilities.csv
/kaggle/input/qureai-headct/reads.csv
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000110.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000113.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000067.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000028.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000068.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000152.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000009.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000147.dcm
/kaggle/input/qureai-headct/qct03/CQ500CT156 CQ500CT156/Unknown Study/CT PLAIN THIN/CT000125.dcm
/kaggle/input/qureai-headct/qct0

KeyboardInterrupt: 

In [2]:
import pandas as pd
import pydicom
import pydicom.dataset
import re
import glob

# load and inspect reads.csv
reads = pd.read_csv("/kaggle/input/qureai-headct/reads.csv")
reads['ICH-sum']		= reads[['R1:ICH', 'R2:ICH', 'R3:ICH']].sum(1)
reads['ICH-majority']	= (reads['ICH-sum'] >= 2).astype(int)
reads['ICH-soft']		= reads['ICH-sum'] / 3
compact_reads = reads[['name', 'ICH-sum', 'ICH-majority', 'ICH-soft']]
compact_reads.to_csv('compact_reads.csv', index=False)
compact_reads.head()

,name,ICH-sum,ICH-majority,ICH-soft
0,CQ500-CT-427,3,1,1.0
1,CQ500-CT-181,3,1,1.0
2,CQ500-CT-99,0,0,0.0
3,CQ500-CT-47,0,0,0.0
4,CQ500-CT-195,0,0,0.0


In [7]:
# TEST directories
import glob
print(glob.glob(f"/kaggle/input/qureai-headct/*"))

['/kaggle/input/qureai-headct/qct03', '/kaggle/input/qureai-headct/qct06', '/kaggle/input/qureai-headct/qct08', '/kaggle/input/qureai-headct/qct10', '/kaggle/input/qureai-headct/qct07', '/kaggle/input/qureai-headct/qct02', '/kaggle/input/qureai-headct/qct17', '/kaggle/input/qureai-headct/qct05', '/kaggle/input/qureai-headct/qct11', '/kaggle/input/qureai-headct/qct19', '/kaggle/input/qureai-headct/qct01', '/kaggle/input/qureai-headct/qct16', '/kaggle/input/qureai-headct/qct13', '/kaggle/input/qureai-headct/qct14', '/kaggle/input/qureai-headct/qct15', '/kaggle/input/qureai-headct/qct12', '/kaggle/input/qureai-headct/qct09', '/kaggle/input/qureai-headct/qct18', '/kaggle/input/qureai-headct/qct04', '/kaggle/input/qureai-headct/prediction_probabilities.csv', '/kaggle/input/qureai-headct/reads.csv']


In [8]:
DATA_ROOT = "/kaggle/input/qureai-headct"
PID = re.compile(r"CQ500CT(\d{1,3})")  # 0-999
records = []

for fp in glob.glob(f"{DATA_ROOT}/qct*/*/*/*", recursive=True):
    folder = PID.search(fp)  # folder names
    if not folder:
        continue
    pid = int(folder.group(1))  # numbers
    print(f"CQ500CT{pid}")  # not zero padded - main patient folder
    DIR = fp
    for dcm_file in glob.glob(f"{DIR}/*.dcm", recursive=True):
        ds: pydicom.dataset.FileDataset = pydicom.dcmread(
            dcm_file, stop_before_pixels=True
        )  ## Metadata only
        records.append(
            {
                "name": f"CQ500-CT-{pid}",
                "series_uid": ds.SeriesInstanceUID,
                "instance_num": ds.get("InstanceNumber", -1),
                "path": dcm_file,
                "slice_thick_mm": float(ds.get("SliceThickness", -1)),
                "series_desc": ds.get("SeriesDescription", ""),
            }
        )
manifest = pd.DataFrame(records).sort_values(["name", "series_uid", "instance_num"])
manifest.to_parquet("cq500ct_manifest.parquet", index=False)
print("Wrote", len(manifest), "rows")

CQ500CT156
CQ500CT156
CQ500CT144
CQ500CT15
CQ500CT15
CQ500CT162
CQ500CT162
CQ500CT162
CQ500CT162
CQ500CT162
CQ500CT158
CQ500CT158
CQ500CT149
CQ500CT149
CQ500CT149
CQ500CT149
CQ500CT149
CQ500CT149
CQ500CT154
CQ500CT154
CQ500CT163
CQ500CT157
CQ500CT146
CQ500CT146
CQ500CT146
CQ500CT146
CQ500CT150
CQ500CT150
CQ500CT150
CQ500CT150
CQ500CT151
CQ500CT151
CQ500CT165
CQ500CT165
CQ500CT165
CQ500CT145
CQ500CT145
CQ500CT145
CQ500CT148
CQ500CT148
CQ500CT148
CQ500CT148
CQ500CT148
CQ500CT148
CQ500CT148
CQ500CT167
CQ500CT155
CQ500CT155
CQ500CT166
CQ500CT166
CQ500CT166
CQ500CT166
CQ500CT152
CQ500CT152
CQ500CT164
CQ500CT164
CQ500CT164
CQ500CT164
CQ500CT153
CQ500CT153
CQ500CT161
CQ500CT159
CQ500CT159
CQ500CT159
CQ500CT159
CQ500CT147
CQ500CT147
CQ500CT14
CQ500CT14
CQ500CT224
CQ500CT229
CQ500CT229
CQ500CT229
CQ500CT22
CQ500CT22
CQ500CT237
CQ500CT237
CQ500CT237
CQ500CT237
CQ500CT235
CQ500CT235
CQ500CT21
CQ500CT21
CQ500CT21
CQ500CT220
CQ500CT220
CQ500CT232
CQ500CT232
CQ500CT234
CQ500CT234
CQ500CT234
CQ500CT2

In [15]:
# Imports
import cv2
import numpy as np
from pandas import DataFrame
import torch
import pydicom
import pydicom.dataset
from pydicom.pixel_data_handlers.util import _apply_modality_lut


# Class
class CQ500Dataset(torch.utils.data.Dataset):
    """
    Connects `cq500ct_manifest.parquet` and `label.csv` and turns into Pytorch `tensors`.
    Returns x (shape), y (torch tensor float32) could be used as tran_ds.
    """

    def __init__(
        self, manifest_df: DataFrame, labels_df: DataFrame, transform=None
    ) -> None:
        """
        manifest_df is the `parquet` file created from the whole dataset.
        labels_df is the `csv` file created from `reads.csv`.
        """
        self.mf = manifest_df
        self.lbl = labels_df.set_index("name")  # name as index
        # self.ids become the master ids list, serving the pipeline.
        # find the unique and pixel + labels scans
        self.ids = self.lbl.index.intersection(self.mf["name"].unique())
        self.tf = transform

    def __len__(self) -> int:
        """Report the number of studies (not slices) as `int`."""
        return len(self.ids)

    def __getitem__(self, idx) -> tuple:
        study = self.ids[idx]  ## Pick a study (not slices)
        df = self.mf[
            self.mf["name"] == study
        ]  ## All the rows belonging to this study (chosen index) -> single patient row in manifest
        sid = df["series_uid"].iloc[0]  ## Pick one series #! ???
        slices = df[
            df["series_uid"] == sid
        ]  ## All slices (rows) for the selected series within the study
        volume = [to_windowed_tensor(pydicom.dcmread(p)) for p in slices["path"]]
        x = torch.stack(
            volume
        )  ## Stack per-slice tensors into a 4D batch [num_slice, num_chan, h, w]
        y = self.lbl.loc[
            study, "ICH-majority"
        ]  ## Target scalar label (soft or majority)
        if self.tf:
            x = self.tf(x)
        return x, torch.tensor(y, dtype=torch.float32)


# Helper
# DICOM to Tensor
def to_windowed_tensor(
    ds: pydicom.dataset.FileDataset,
    windows: list[tuple[int, int]] = None,
    out_size: tuple[int, int] = None,
    dtype: torch.dtype = torch.float32,
) -> torch.tensor:
    """
    Helper: convert one DICOM slice into a torch.Tensor [C x H x W]
    ds			= single CQ500 DICOM slice.
    windows		= CT window to apply. Each tuple becomes one output channel.
                    Default = Brain, Subdural, Bone.
    out_size	= (H, W) to resize the slice.
    dtype		= final tensor percision (float32 recommended).

    Returns		= torch.tensor (shape = C x H x W with normalized values)
    """
    ## Handle None args
    if windows is None:
        windows = [(40, 80), (80, 200), (600, 2800)]
    if out_size is None:
        out_size = (256, 256)

    ## Raw values > Hounsfield units (HU)
    hu: np.ndarray = _apply_modality_lut(ds.pixel_array, ds).astype(np.int16)
    if out_size is not None and hu.shape != out_size:
        hu = cv2.resize(hu, out_size[::-1], interpolation=cv2.INTER_LINEAR)

    ## Window / Level > 0-1 float per channel
    ## 3 channels to feed into model
    chans: list[np.ndarray] = []
    for level, width in windows:
        level: int
        width: int
        lower: int = level - (width // 2)
        upper: int = level + (width // 2)
        img_clipped: np.ndarray = np.clip(hu, lower, upper)
        img_norm: np.ndarray = (img_clipped - lower) / float(width)  # 0 - 1
        chans.append(img_norm.astype(np.float32))

    ## Stack and convert to tensor
    arr: np.ndarray = np.stack(chans, axis=0)  # C x H x W
    tensor: torch.Tensor = torch.from_numpy(arr).type(dtype)

    return tensor


In [19]:
import pandas as pd
pq = pd.read_parquet("cq500ct_qct19_manifest.parquet")
pq["name"] = pq["name"].astype(str)

compact_reads = pd.read_csv("compact_reads.csv")

ds = CQ500Dataset(manifest_df=pq, labels_df=compact_reads, transform=None)
print(f"Studies available: {len(ds)}")	# available studies

Studies available: 473


In [28]:
#!pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
#!pip install python-gdcm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.8 MB/s eta 0:00:0000:0100:01


In [32]:
metadata_df = pq.merge(
    compact_reads[['name', 'ICH-majority']],
    on='name',
    how='left',
    validate='many_to_one',
    indicator=True
)
metadata_df.to_parquet("cq500ct_metadata.parquet", index=False)

In [35]:
## Shuffle split to prevent leakage. Split by patient "name"
from sklearn.model_selection import GroupShuffleSplit
splitter = GroupShuffleSplit(
    test_size=0.2,
    n_splits=1,
    random_state=42
)
train_idx, val_idx = next(splitter.split(X=metadata_df, y=metadata_df["ICH-majority"], groups=metadata_df["name"]))

In [38]:
# Check leakage between train and validation sets
assert set(metadata_df.iloc[train_idx].name).isdisjoint(metadata_df.iloc[val_idx].name), "Leakage"